In [2]:
import requests

# Using Epic Test Server

In [9]:
EPIC_BASE_URL = "https://connectathon.epic.com/interconnect-fhir-oauth/api/FHIR/R4/"
TEST_PATIENT_ID = "e0VZT8JQnSYBU1u97nG6L3A3"

## Scenario 0

In [5]:
DISCOVERY_PATH = ".well-known/smart-configuration"

In [6]:
epic_config = requests.get(EPIC_BASE_URL + DISCOVERY_PATH)

- Payload includes `permission_v2` and `authorize-post` capabilities ✔️
- `code_challenge_methods_supported` includes `S256` ✔️
- `introscpection_endpoint` included ✔️

In [7]:
epic_config.text

'<discovery xmlns="http://hl7.org/fhir"><authorization_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/authorize" /><token_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/token" /><token_endpoint_auth_methods_supported value="client_secret_post" /><token_endpoint_auth_methods_supported value="client_secret_basic" /><token_endpoint_auth_methods_supported value="private_key_jwt" /><scopes_supported value="epic.scanning.dmsusername" /><scopes_supported value="fhirUser" /><scopes_supported value="launch" /><scopes_supported value="openid" /><scopes_supported value="profile" /><response_types_supported value="code" /><introspection_endpoint value="https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/introspect" /><capabilities value="launch-ehr" /><capabilities value="launch-standalone" /><capabilities value="client-public" /><capabilities value="client-confidential-symmetric" /><capabilities value="context-banner" /><capabil

## Scenario 1

In [8]:
authorize_url = "https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/authorize"
token_url = "https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/token"

redirect_uri = "http://localhost:8000/index.html"

client_id = "GranularConnect2021Scenario2"

scope = "patient/Observation.rs"

authorization_redirect_url = authorize_url + '?response_type=code&client_id=' + client_id + '&redirect_uri=' + redirect_uri + '&scope=' + scope
    
print("Navigate to: ", authorization_redirect_url + "\n")
    
authorization_code = input('authz code: ')
    
headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
data = {'grant_type': 'authorization_code', 'code': authorization_code, 'redirect_uri': redirect_uri, 'client_id': client_id}
access_token_response = requests.post(token_url, data=data, headers=headers)

print("\nScopes Granted: " + access_token_response.json()['scope'])

Navigate to:  https://connectathon.epic.com/Interconnect-Fhir-OAuth/oauth2/authorize?response_type=code&client_id=GranularConnect2021Scenario2&redirect_uri=http://localhost:8000/index.html&scope=patient/Observation.rs

authz code: eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOiJHZW5lcmljLUhzaSIsImNsaWVudF9pZCI6IkdyYW51bGFyQ29ubmVjdDIwMjFTY2VuYXJpbzIiLCJlcGljLmVjaSI6InVybjplcGljOkN1cnJlbnQtQ29ubmVjdGF0aG9uIiwiZXBpYy5tZXRhZGF0YSI6Il9RMEhTZ01wZkl4UGhkaW13QWJMVlhpb2NoZWFCSTZFWWt2OFdRM05RNWlHazZXSFprWFFSTFhkcUtMSVN4YUtvUnNpQ1BZWnFreWJuOG94QmhqaHREUkwxbDVLdXEtRWdIbFMxTEZoNHVTT0FmNHZLcGI2WUlTNlV1V3k3WGxRIiwiZXBpYy50b2tlbnR5cGUiOiJjb2RlIiwiZXhwIjoxNjEwNzM3NTc0LCJpYXQiOjE2MTA3MzcyNzQsImlzcyI6IkdlbmVyaWMtSHNpIiwianRpIjoiOTZjMDVhMjgtMDQ3OS00OTZiLTk0YWQtZGQzZmI3NmM5NmVmIiwibmJmIjoxNjEwNzM3Mjc0LCJzdWIiOiJlYVU4aS1KNmtYOXFHcmxxb2VOd290dzMifQ.Ee_yxipFie0mm27yBCEYrNZB-A4lISJIhxfM8Vc4WmdSQGeVHJj4Hw9rg7S3N_3P-CmxvjDWAHxUu_PtMMlDih6-XvKGt6K_dLd2VLxaZ2U3_ohGgX_NELbuiVXHpN3F6JU15MAmKNJd2-NbGmRN6ojkXFnczY1tv

`Epic` does not support scoping a client for all Observations:

In [14]:
headers = {"Authorization" : "Bearer " + access_token_response.json()['access_token']}
data_request_response = requests.get("{}Observation?patient={}".format(EPIC_BASE_URL, TEST_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [400]>


But requesting a specific category like `vital-signs` works:

In [13]:
headers = {"Authorization" : "Bearer " + access_token_response.json()['access_token']}
data_request_response = requests.get("{}Observation?patient={}&category=vital-signs".format(EPIC_BASE_URL, TEST_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [200]>
